In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)

import numpy as np

import os

import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,ConfusionMatrixDisplay
import sklearn
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score
import numpy as np
pd.set_option('display.max_rows', 500)

from transformers import BertTokenizer, BertForMaskedLM
import torch
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,log_loss,precision_score,recall_score,classification_report
from transformers import pipeline

2022-06-13 03:39:42.104520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-13 03:39:42.104541: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df_long=pd.read_csv("../data/train_val_keywords.csv")

In [3]:
preprompt="Does this paper tackle a social problem?\n"

In [4]:
df_long.abstract=df_long.abstract.fillna("")

In [5]:
df_long.abstract=df_long.abstract.replace("{","",regex=True).replace("}","",regex=True)
df_long.title=df_long.title.replace("{","",regex=True).replace("}","",regex=True)

In [13]:
df_long=df_long.assign(statement=preprompt+"Title: "+df_long.title+"\nAbstract: "+df_long.abstract
                       +"\nDoes this paper tackle a social problem? Answer \"Yes\" or \"No\": \"")

In [7]:
model = pipeline('fill-mask', model='bert-large-uncased',top_k=20)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
for i,d in df_long.iterrows():
    print(d['statement'])
    input_prompt=d['statement']+" [MASK]"
    predicted_blanks = model(input_prompt)
    # print the completion

    dict_uniques={}
    dict_norm={}
    for p in predicted_blanks:
        p_modified=p['token_str'].lower().lstrip(' ')
        if p_modified in dict_uniques:
            dict_uniques[p_modified]=dict_uniques[p_modified]+p['score']
        else:
            dict_uniques[p_modified]=p['score']

    if ('no' in dict_uniques.keys()) and ('yes' in dict_uniques.keys()):
        dict_norm={'no':dict_uniques['no'],'yes':dict_uniques['yes']}
    elif ('no' in dict_uniques.keys()):
        dict_norm={'no':dict_uniques['no'],'yes':0}
    elif ('yes' in dict_uniques.keys()):
        dict_norm={'no':0,'yes':dict_uniques['yes']}
    try:
        factor=1.0/sum(dict_norm.values())
        for k in dict_norm:
            dict_norm[k] = dict_norm[k]*factor    

        df_long.loc[i,'proba_1']=dict_norm['yes']
        df_long.loc[i,'proba_0']=dict_norm['no']

        #df_long.loc[i,'BERT_response']=predicted_blanks
        print("Yes: ",dict_norm['yes'],"No: ",dict_norm['no'])
        print("Label: ",d['label'])
        print("################################")
    except:
        continue

Does this paper tackle a social problem?
Title: Syntactic Data Augmentation Increases Robustness to Inference Heuristics
Abstract: Pretrained neural models such as BERT, when fine-tuned to perform natural language inference (NLI), often show high accuracy on standard datasets, but display a surprising lack of sensitivity to word order on controlled challenge sets. We hypothesize that this issue is not primarily caused by the pretrained model's limitations, but rather by the paucity of crowdsourced NLI examples that might convey the importance of syntactic structure at the fine-tuning stage. We explore several methods to augment standard training sets with syntactically informative examples, generated by applying syntactic transformations to sentences from the MNLI corpus. The best-performing augmentation method, subject/object inversion, improved BERT's accuracy on controlled examples that diagnose sensitivity to word order from 0.28 to 0.73, without affecting performance on the MNLI t

Yes:  0.6886825502054106 No:  0.3113174497945895
Label:  0
################################
Does this paper tackle a social problem?
Title: K2Q: Generating Natural Language Questions from Keywords with User Refinements
Abstract: 
Does this paper tackle a social problem? Answer "Yes" or "No": "
Yes:  1.0 No:  0.0
Label:  0
################################
Does this paper tackle a social problem?
Title: Semantic Vectors: a Scalable Open Source Package and Online Technology Management Application
Abstract: This paper describes the open source SemanticVectors package that efficiently creates semantic vectors for words and documents from a corpus of free text articles. We believe that this package can play an important role in furthering research in distributional semantics, and (perhaps more importantly) can help to significantly reduce the current gap that exists between good research results and valuable applications in production software. Two clear principles that have guided the creat

Yes:  1.0 No:  0.0
Label:  0
################################
Does this paper tackle a social problem?
Title: Finding the Origin of a Translated Historical Document
Abstract: 
Does this paper tackle a social problem? Answer "Yes" or "No": "
Yes:  1.0 No:  0.0
Label:  0
################################
Does this paper tackle a social problem?
Title: Exploiting a Probabilistic Hierarchical Model for Generation
Abstract: 
Does this paper tackle a social problem? Answer "Yes" or "No": "
Yes:  1.0 No:  0.0
Label:  0
################################
Does this paper tackle a social problem?
Title: The ICSI Meeting Recorder Dialog Act (MRDA) Corpus
Abstract: 
Does this paper tackle a social problem? Answer "Yes" or "No": "
Yes:  1.0 No:  0.0
Label:  0
################################
Does this paper tackle a social problem?
Title: One Language to rule them all: modelling Morphological Patterns in a Large Scale Italian Lexicon with SWRL
Abstract: 
Does this paper tackle a social problem? Answer

Yes:  0.9999999999999999 No:  0.0
Label:  0
################################
Does this paper tackle a social problem?
Title: Natural Language Reasoning using Coq: Interaction and Automation
Abstract: Dans cet article, nous pr\'esentons une utilisation des assistants des preuves pour traiter l'inf\'erence en Language Naturel (NLI). D' abord, nous proposons d'utiliser les theories des types modernes comme langue dans laquelle traduire la s\'emantique du langage naturel. Ensuite, nous impl\'ementons cette s\'emantique dans l'assistant de preuve Coq pour raisonner sur ceux-ci. En particulier, nous \'evaluons notre proposition sur un sous-ensemble de la suite de tests FraCas, et nous montrons que 95.2\% des exemples peuvent \^etre correctement pr\'edits. Nous discutons ensuite la question de l'automatisation et il est d\'emontr\'e que le langage de tactiques de Coq permet de construire des tactiques qui peuvent automatiser enti\`erement les preuves, au moins pour les cas qui nous int\'eress

Yes:  1.0 No:  0.0
Label:  0
################################
Does this paper tackle a social problem?
Title: Automaton-based Parsing for Lexicalised Grammars
Abstract: In wide-coverage lexicalized grammars many of the elementary structures have substructures in common. This means that during parsing some of the computation associated with different structures is duplicated. This paper explores ways in which the grammar can be precompiled into finite state automata so that some of this shared structure results in shared computation at run-time.
Does this paper tackle a social problem? Answer "Yes" or "No": "
Yes:  0.40580924311356137 No:  0.5941907568864386
Label:  0
################################
Does this paper tackle a social problem?
Title: SWATCS65: Sentiment Classification Using an Ensemble of Class Projects
Abstract: 
Does this paper tackle a social problem? Answer "Yes" or "No": "
Yes:  1.0 No:  0.0
Label:  0
################################
Does this paper tackle a social pr

Yes:  0.7357321422753841 No:  0.2642678577246158
Label:  0
################################
Does this paper tackle a social problem?
Title: On the Complementarity between Pre-Training and Back-Translation for Neural Machine Translation
Abstract: Pre-training (PT) and back-translation (BT) are two simple and powerful methods to utilize monolingual data for improving the model performance of neural machine translation (NMT). This paper takes the first step to investigate the complementarity between PT and BT. We introduce two probing tasks for PT and BT respectively and find that PT mainly contributes to the encoder module while BT brings more benefits to the decoder. Experimental results show that PT and BT are nicely complementary to each other, establishing state-of-the-art performances on the WMT16 English-Romanian and English-Russian benchmarks. Through extensive analyses on sentence originality and word frequency, we also demonstrate that combining Tagged BT with PT is more helpful

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors


Yes:  0.9999999999999999 No:  0.0
Label:  0
################################
Does this paper tackle a social problem?
Title: La perception des s\'equences consonantiques non-natives par les locuteurs monolingues de mandarin (Perception of non-native consonant sequences by Mandarin monolingual speakers)
Abstract: Cette \'etude examine le r\^ole de la structure phonotactique native et des facteurs phon\'etiques dans la perception des s\'equences consonantiques non-natives. Des locuteurs monolingues de mandarin ont \'et\'e test\'es dans les deux exp\'eriences suivantes: dans la premi\`ere exp\'erience, les locuteurs ont du d\'ecider s'ils entendaient une voyelle entre deux consonnes en \'ecoutant des s\'equences intervocaliques-CC (akta) et leurs contr\^oles CVC (akata). Les participants mandarins monolingues ont tendance \`a percevoir une voyelle entre deux consonnes dans les deux s\'equences CC et CVC. Mais le pourcentage de la voyelle per\ccue varie selon les diff\'erentes s\'equences.

RuntimeError: The size of tensor a (526) must match the size of tensor b (512) at non-singleton dimension 1

In [15]:
dict_uniques

{'.': 0.48027274012565613,
 '?': 0.4616122245788574,
 '!': 0.026248710229992867,
 ';': 0.017631106078624725,
 '|': 0.011219942010939121,
 '"': 0.002135386224836111,
 '...': 0.00042458821553736925,
 "'": 0.00014783229562453926,
 '।': 3.122911584796384e-05,
 '[unk]': 2.2687643649987876e-05,
 '-': 1.4771692804060876e-05,
 '॥': 1.2983095984964166e-05,
 ']': 1.2935525774082635e-05,
 ':': 1.0214113899564836e-05,
 '}': 8.818041351332795e-06,
 ')': 4.571681984089082e-06,
 ',': 4.232653736835346e-06,
 'if': 3.419616405153647e-06,
 'and': 3.273455831731553e-06,
 '(': 3.2142281725100474e-06}

df_long=pd.read_csv("predictions_bert_1.csv")

In [24]:
df_long=df_long.assign(human_response_binary=np.where(df_long['human.response']>0.5,1,0))

In [25]:
df_long=df_long.assign(study_aux=np.where(df_long.study.isin(['deli.lines','snack.lines']),'lines',df_long.study))

In [26]:
df_long.proba_1.describe()

count    148.000000
mean       0.494062
std        0.108784
min        0.343827
25%        0.422775
50%        0.458609
75%        0.531043
max        0.775715
Name: proba_1, dtype: float64

## alternative response

In [27]:
df_long=df_long.assign(GPT3_response_probas_binary=np.where(df_long.proba_1>0.5,1,0))


In [28]:
df_long=df_long.assign(correct_probas=np.where(df_long.human_response_binary==df_long.GPT3_response_probas_binary,1,0))

In [29]:
df_long.correct_probas.mean()

0.5405405405405406

In [30]:
df_long.groupby(['study_aux']).correct_probas.mean()

study_aux
blue.house    0.537037
cannonball    0.571429
lines         0.530303
Name: correct_probas, dtype: float64

In [31]:
print("MAE: ",mean_absolute_error(df_long['human.response'], df_long.proba_1))

print("accuracy: ",f1_score(df_long.loc[:,'human_response_binary'], df_long.loc[:,'GPT3_response_probas_binary'],average='micro'))

MAE:  0.270138318109218
accuracy:  0.5405405405405406


In [32]:
## gamma long
print(classification_report(df_long.loc[:,'human_response_binary'], df_long.loc[:,'GPT3_response_probas_binary']))

              precision    recall  f1-score   support

           0       0.61      0.67      0.64        90
           1       0.40      0.34      0.37        58

    accuracy                           0.54       148
   macro avg       0.51      0.51      0.50       148
weighted avg       0.53      0.54      0.53       148



In [33]:
for c in df_long['study_aux'].unique():
    print(c)
    dd=classification_report(df_long.loc[df_long['study_aux']==c,'human_response_binary'],
                            df_long.loc[df_long['study_aux']==c,'GPT3_response_probas_binary'],digits=4,output_dict=True)
    print("{0:.2f}".format(100*dd['weighted avg']['f1-score']))

blue.house
57.24
cannonball
50.88
lines
45.16


In [34]:
## domain feat
gen=classification_report(df_long.loc[:,'human_response_binary'],
                            df_long.loc[:,'GPT3_response_probas_binary'],digits=4,output_dict=True)

print("f1w ","{0:.2f}".format(100*gen['weighted avg']['f1-score']))
print("accuracy ","{0:.2f}".format(100*gen['accuracy']))
print("w-precision ","{0:.2f}".format(100*gen['weighted avg']['precision']))
print("w-recall ","{0:.2f}".format(100*gen['weighted avg']['recall']))

f1w  53.33
accuracy  54.05
w-precision  52.91
w-recall  54.05


In [35]:
df_long.GPT3_response_probas_binary.value_counts()

0    98
1    50
Name: GPT3_response_probas_binary, dtype: int64

In [36]:
df_long.to_csv("predictions_bert_1.csv",index=False)

## Cross entropy

In [37]:
from keras.losses import binary_crossentropy
from keras import backend

In [38]:
mean_ce = backend.eval(binary_crossentropy(df_long['human.response'], df_long['proba_1']))
print('Average Cross Entropy: %.3f nats' % mean_ce)

Average Cross Entropy: 0.714 nats


In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)

import numpy as np

import os

import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,ConfusionMatrixDisplay
import sklearn
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score
import numpy as np
pd.set_option('display.max_rows', 500)

from transformers import BertTokenizer, BertForMaskedLM
import torch
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,log_loss,precision_score,recall_score,classification_report
from transformers import pipeline

df_long=pd.read_csv("./short_long_prompts_v2.csv")

preprompt="Read the situation and answer the question.\nSituation: "


df_long=df_long.assign(pre_statement=np.where(df_long.study=='blue.house',"Analyze Hank's action.",
                                             "Analyze that person's action."))

df_long=df_long.assign(statement=preprompt+df_long.prompt_short+"\nQuestion: "+df_long.pre_statement+" Is it ok to break the rule in this situation? Answer by one capitalized word, either \"YES\" or \"NO\"."+"\nAnswer: \"")

df_long=df_long.assign(pre_statement=np.where(df_long.study=='blue.house',"Analyze Hank's action. Is it ok to break the rule in this situation? Answer just \"yes\" or \"no\".",
                                             "Analyze that person's action. Is it ok to break the rule in this situation? Answer just \"yes\" or \"no\"."))

df_long=df_long.assign(statement=preprompt+df_long.prompt_short+"\nQuestion: "+df_long.pre_statement+"\nAnswer: \"")

df_long=df_long.assign(pre_statement=np.where(df_long.study=='blue.house',"Analyze Hank's action. Is it ok to break the rule in this situation? Answer by one capitalized word, either \"YES\" or \"NO\".",
                                             "Analyze that person's action. Is it ok to break the rule in this situation? Answer by one capitalized word, either \"YES\" or \"NO\"."))

df_long=df_long.assign(statement=preprompt+df_long.prompt_short+"\nQuestion: "+df_long.pre_statement+"\nAnswer: \"")

model = pipeline('fill-mask', model='bert-large-uncased',top_k=15)

for i,d in df_long.iterrows():
    print(d['statement'])
    input_prompt=d['statement']+" [MASK]"
    predicted_blanks = model(input_prompt)
    # print the completion

    dict_uniques={}
    dict_norm={}
    for p in predicted_blanks:
        p_modified=p['token_str'].lower().lstrip(' ')
        if p_modified in dict_uniques:
            dict_uniques[p_modified]=dict_uniques[p_modified]+p['score']
        else:
            dict_uniques[p_modified]=p['score']

    if ('no' in dict_uniques.keys()) and ('yes' in dict_uniques.keys()):
        dict_norm={'no':dict_uniques['no'],'yes':dict_uniques['yes']}
    elif ('no' in dict_uniques.keys()):
        dict_norm={'no':dict_uniques['no'],'yes':0}
    elif ('yes' in dict_uniques.keys()):
        dict_norm={'no':0,'yes':dict_uniques['yes']}

    factor=1.0/sum(dict_norm.values())
    for k in dict_norm:
        dict_norm[k] = dict_norm[k]*factor    

    df_long.loc[i,'proba_1']=dict_norm['yes']
    df_long.loc[i,'proba_0']=dict_norm['no']
    
    #df_long.loc[i,'BERT_response']=predicted_blanks
    print("Yes: ",dict_norm['yes'],"No: ",dict_norm['no'])
    print("Human response: ",d['human.response'])
    print("################################")

df_long=pd.read_csv("predictions_bert_1.csv")

df_long=df_long.assign(human_response_binary=np.where(df_long['human.response']>0.5,1,0))

df_long=df_long.assign(study_aux=np.where(df_long.study.isin(['deli.lines','snack.lines']),'lines',df_long.study))

df_long.proba_1.describe()

## alternative response

df_long=df_long.assign(GPT3_response_probas_binary=np.where(df_long.proba_1>0.5,1,0))


df_long=df_long.assign(correct_probas=np.where(df_long.human_response_binary==df_long.GPT3_response_probas_binary,1,0))

df_long.correct_probas.mean()

df_long.groupby(['study_aux']).correct_probas.mean()

print("MAE: ",mean_absolute_error(df_long['human.response'], df_long.proba_1))

print("accuracy: ",f1_score(df_long.loc[:,'human_response_binary'], df_long.loc[:,'GPT3_response_probas_binary'],average='micro'))

## gamma long
print(classification_report(df_long.loc[:,'human_response_binary'], df_long.loc[:,'GPT3_response_probas_binary']))

for c in df_long['study_aux'].unique():
    print(c)
    dd=classification_report(df_long.loc[df_long['study_aux']==c,'human_response_binary'],
                            df_long.loc[df_long['study_aux']==c,'GPT3_response_probas_binary'],digits=4,output_dict=True)
    print("{0:.2f}".format(100*dd['weighted avg']['f1-score']))

## domain feat
gen=classification_report(df_long.loc[:,'human_response_binary'],
                            df_long.loc[:,'GPT3_response_probas_binary'],digits=4,output_dict=True)

print("f1w ","{0:.2f}".format(100*gen['weighted avg']['f1-score']))
print("accuracy ","{0:.2f}".format(100*gen['accuracy']))
print("w-precision ","{0:.2f}".format(100*gen['weighted avg']['precision']))
print("w-recall ","{0:.2f}".format(100*gen['weighted avg']['recall']))

df_long.GPT3_response_probas_binary.value_counts()

df_long.to_csv("predictions_bert_1.csv",index=False)

## Cross entropy

from keras.losses import binary_crossentropy
from keras import backend

mean_ce = backend.eval(binary_crossentropy(df_long['human.response'], df_long['proba_1']))
print('Average Cross Entropy: %.3f nats' % mean_ce)